# Preparação do ambiente

## Instalação de dependências

In [71]:
# !pip install pandas
# !pip install python-dotenv
# !pip install requests
# !pip install openai==0.28.1

## Importação das bibliotecas

In [ ]:
# Importação das bibliotecas
import os
import openai
from dotenv import load_dotenv

## Carregar variáveis de ambiente

In [73]:
load_dotenv()

True

## Configuração do OpenRouter

In [74]:
openai.api_key = os.getenv("OPENROUTER_API_KEY") # API Key do OpenRouter
openai.api_base = "https://openrouter.ai/api/v1" # URL da API do OpenRouter

In [75]:
# Modelo do OpenRouter
model_name = "meta-llama/llama-3.1-8b-instruct:free"

# Obter resposta do LLM

In [76]:
# Função para enviar uma pergunta e obter uma resposta do modelo
def get_chatbot_response(messages, system_prompt="", temperature=0, model=model_name):
    # Type check do messages
    if not isinstance(messages, list):
        raise TypeError("messages deve ser uma lista")

    # Lista de mensagens pro modelo
    input_messages = []
    for message in messages:
        # Adicionar dicionário à lista
        input_messages.append({
            "role": message["role"], # Quem está falando (usuário ou sistema)
            "content": message["content"] # Conteúdo da mensagem
        })

    response = openai.ChatCompletion.create(
        model=model, # Modelo de IA
        # Lista de mensagens
        messages=input_messages,
        # Temperatura: quantidade de aleatoriedade na resposta
        temperature=temperature, # 0 pois queremos resultados concretos
        top_p=0.8,
        max_tokens=2_000, # Limite de tokens na resposta
        # Token: unidade de medida do modelo de IA (palavras, sub-palavras, caracteres, ...)
    ).choices[0].message.content.strip()

    return response # Retornar resposta ao usuário

In [77]:
# Teste da função
msgs_usuario = [{"role": "user", "content": "Qual é a capital da Austrália?"}]
resposta = get_chatbot_response(msgs_usuario)
print(f"Usuário: {msgs_usuario[0]['content']}\nChatBot: {resposta}")

Usuário: Qual é a capital da Austrália?
ChatBot: A capital da Austrália é Canberra.


# Engenharia de Prompt

## Structured output
Armazena a resposta do chatbot em um formato estruturado (como CSV ou JSON) para poder ser utilizado em outros sistemas/aplicações (como React Native).

### System Prompt
Prompt dado ao LLM que define seu comportamento.

In [78]:
system_prompt = """
Você é um assistente prestativo que responde perguntas sobre capitais de países.

Seu output deve ser um formato JSON estruturado exatamente como no exemplo abaixo. Você não tem permissão de escrever qualquer outra coisa além do objeto JSON.
[
    {
        "country": O país que o usuário perguntou a capital,
        "capital": A capital do país que o usuário perguntou,
    }
]
"""

In [83]:
# Teste do system_prompt
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Qual é a capital da Nova Zelândia?"}]
resposta = get_chatbot_response(messages, system_prompt=system_prompt)
print(resposta)
print("Data type:", type(resposta))

[
    {
        "country": "Nova Zelândia",
        "capital": "Wellington"
    }
]
Data type: <class 'str'>


In [80]:
# Teste 2
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Qual é a capital da França e do Brasil?"}]
print(get_chatbot_response(messages, system_prompt=system_prompt))

[
    {
        "country": "Fran\u00e7a",
        "capital": "Paris"
    },
    {
        "country": "Brasil",
        "capital": "Bras\u00edlia"
    }
]
